In [1]:
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

train = pd.read_csv("/content/drive/MyDrive/model-citizens/data/Training_TriGuard.csv")
test  = pd.read_csv("/content/drive/MyDrive/model-citizens/data/Testing_TriGuard.csv")

train = train.dropna(subset=['subrogation'])

X_train = train.drop(columns=["subrogation"]).copy()
y_train = train["subrogation"].copy()
X_test  = test.copy()

class Preprocessor:
    def __init__(self):
        self.id_column = ['claim_number']
        self.min_driver_age = 14
        self.max_driver_age = 100

        self.categorical_cols_ = None
        self.label_encoders = {}
        self.income_q25 = None
        self.income_q75 = None

    def _coerce_and_clean(self, df):
        df = df.copy()
        df['claim_date'] = pd.to_datetime(df['claim_date'], errors='coerce')
        df['claim_year'] = df['claim_date'].dt.year
        df.loc[(df['year_of_born'] < 1900) | (df['year_of_born'] > 2025), 'year_of_born'] = np.nan
        future_mask = df['vehicle_made_year'] > df['claim_year']
        df.loc[future_mask, 'vehicle_made_year'] = np.nan
        return df

    def _create_features(self, df):
        df = df.copy()
        df['is_multi_vehicle_clear'] = (df['accident_type'] == 'multi_vehicle_clear').astype(int)
        df['is_multi_vehicle_unclear'] = (df['accident_type'] == 'multi_vehicle_unclear').astype(int)
        df['is_single_car'] = (df['accident_type'] == 'single_car').astype(int)

        df['has_recovery_target'] = (df['is_multi_vehicle_clear'] | df['is_multi_vehicle_unclear']).astype(int)

        df['recovery_case_clarity'] = 0
        df.loc[df['is_multi_vehicle_clear'] == 1, 'recovery_case_clarity'] = 3
        df.loc[df['is_multi_vehicle_unclear'] == 1, 'recovery_case_clarity'] = 1

        df['witness_present'] = df['witness_present_ind']
        df['evidence_none'] = ((df['witness_present'] == 0) & (df['policy_report_filed_ind'] == 0)).astype(int)
        df['evidence_weak'] = (
            ((df['witness_present'] == 1) & (df['policy_report_filed_ind'] == 0)) |
            ((df['witness_present'] == 0) & (df['policy_report_filed_ind'] == 1))
        ).astype(int)
        df['evidence_strong'] = ((df['witness_present'] == 1) & (df['policy_report_filed_ind'] == 1)).astype(int)
        df['evidence_very_strong'] = (
            (df['witness_present'] == 1) &
            (df['policy_report_filed_ind'] == 1) &
            (df['liab_prct'] < 20)
        ).astype(int)

        df['not_at_fault'] = (df['liab_prct'] < 10).astype(int)
        df['minimal_fault'] = ((df['liab_prct'] >= 10) & (df['liab_prct'] <= 20)).astype(int)
        df['shared_fault'] = (df['liab_prct'] > 20).astype(int)

        df['driver_age'] = df['claim_year'] - df['year_of_born']
        bad_age = (df['driver_age'] < self.min_driver_age) | (df['driver_age'] > self.max_driver_age)
        df.loc[bad_age, 'driver_age'] = np.nan
        df['young_driver_18_25'] = ((df['driver_age'] >= 18) & (df['driver_age'] <= 25)).astype(int)
        df['adult_driver_26_45'] = ((df['driver_age'] >= 26) & (df['driver_age'] <= 45)).astype(int)
        df['middle_age_driver_46_65'] = ((df['driver_age'] >= 46) & (df['driver_age'] <= 65)).astype(int)
        df['senior_driver_65plus'] = (df['driver_age'] > 65).astype(int)

        df['driving_experience'] = df['driver_age'] - df['age_of_DL']
        df.loc[df['driving_experience'] < 0, 'driving_experience'] = np.nan
        df['novice_driver'] = (df['driving_experience'] < 2).astype(int)
        df['experienced_2_5y'] = ((df['driving_experience'] >= 2) & (df['driving_experience'] <= 5)).astype(int)
        df['experienced_5_10y'] = ((df['driving_experience'] > 5) & (df['driving_experience'] <= 10)).astype(int)
        df['veteran_driver'] = (df['driving_experience'] > 10).astype(int)

        df['vehicle_age'] = df['claim_year'] - df['vehicle_made_year']
        df['brand_new_vehicle'] = (df['vehicle_age'] <= 1).astype(int)
        df['new_vehicle_2_3y'] = ((df['vehicle_age'] >= 2) & (df['vehicle_age'] <= 3)).astype(int)
        df['mid_age_vehicle_4_7y'] = ((df['vehicle_age'] >= 4) & (df['vehicle_age'] <= 7)).astype(int)
        df['old_vehicle_8_12y'] = ((df['vehicle_age'] >= 8) & (df['vehicle_age'] <= 12)).astype(int)
        df['very_old_vehicle'] = (df['vehicle_age'] > 12).astype(int)

        df['via_broker'] = (df['channel'] == 'Broker').astype(int)
        df['via_online'] = (df['channel'] == 'Online').astype(int)
        df['via_phone']  = (df['channel'] == 'Phone').astype(int)
        df['channel_good_documentation'] = df['channel'].isin(['Broker', 'Online']).astype(int)

        df['low_income'] = np.nan
        df['middle_income'] = np.nan
        df['high_income'] = np.nan

        df['has_high_education'] = df['high_education_ind']
        df['recent_address_change'] = df['address_change_ind']
        df['home_owner'] = (df['living_status'] == 'Own').astype(int)
        df['renter'] = (df['living_status'] == 'Rent').astype(int)
        df['contact_info_available'] = df['email_or_tel_available']
        df['in_network_repair'] = (df['in_network_bodyshop'] == 'yes').astype(int)
        df['out_of_network_repair'] = (df['in_network_bodyshop'] == 'no').astype(int)

        liability_score = np.sqrt((100 - df['liab_prct']) / 100.0)
        evidence_score  = (df['evidence_none'] * 0.0 +
                           df['evidence_weak'] * 0.5 +
                           df['evidence_strong'] * 0.8 +
                           df['evidence_very_strong'] * 1.0)
        clarity_score = df['recovery_case_clarity'] / 3.0
        info_score = df['channel_good_documentation'] * 0.7 + df['contact_info_available'] * 0.3

        weights = np.array([0.35, 0.25, 0.20, 0.15, 0.05])
        parts = np.vstack([
            liability_score,
            df['has_recovery_target'],
            evidence_score,
            clarity_score,
            info_score
        ])
        df['recovery_feasibility_score'] = (parts * weights.reshape(-1,1)).sum(axis=0)

        df['high_subrogation_potential'] = (
            (df['liab_prct'] < 20) &
            (df['has_recovery_target'] == 1) &
            (df['evidence_strong'] == 1) &
            (df['recovery_feasibility_score'] > 0.7)
        ).astype(int)
        df['likely_no_subrogation'] = (
            (df['liab_prct'] > 50) |
            (df['is_single_car'] == 1) |
            (df['evidence_none'] == 1)
        ).astype(int)
        df['potential_subrogation_case'] = (df['high_subrogation_potential'] == 1).astype(int)

        df = df.drop(columns=self.id_column, errors='ignore')
        return df

    def fit(self, df):
        df = self._coerce_and_clean(df.copy())
        # 分位数（只用训练集）
        self.income_q25 = df['annual_income'].quantile(0.25)
        self.income_q75 = df['annual_income'].quantile(0.75)

        df = self._create_features(df)

        self.categorical_cols_ = list(df.select_dtypes(include=['object']).columns)

        self.label_encoders.clear()
        for col in self.categorical_cols_:
            le = LabelEncoder()
            le.fit(df[col].astype(str))
            self.label_encoders[col] = le
        return self

    def transform(self, df):
        df = self._coerce_and_clean(df.copy())
        df = self._create_features(df)

        q25, q75 = self.income_q25, self.income_q75
        df['low_income']    = (df['annual_income'] <= q25).astype(int)
        df['middle_income'] = ((df['annual_income'] > q25) & (df['annual_income'] <= q75)).astype(int)
        df['high_income']   = (df['annual_income'] > q75).astype(int)

        for col, le in self.label_encoders.items():
            df[col] = le.transform(df[col].astype(str))

        df = df.drop(columns=['claim_date'], errors='ignore')
        return df

    def fit_transform(self, df):
        return self.fit(df).transform(df)

pre = Preprocessor()
X_train_proc = pre.fit_transform(X_train)
X_test_proc  = pre.transform(X_test)

X_test_proc = X_test_proc.reindex(columns=X_train_proc.columns, fill_value=0)


model = XGBClassifier(
    n_estimators=800,
    learning_rate=0.06,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    eval_metric="auc"
)

model.fit(X_train_proc, y_train)


test_pred_proba = model.predict_proba(X_test_proc)[:, 1]
test_pred_label = (test_pred_proba >= 0.5).astype(int)

prediction = pd.DataFrame({
    "claim_number": test["claim_number"],
    "subrogation": test_pred_label
})

prediction.to_csv("TriGuard_prediction.csv", index=False)

print(prediction.head())



In [25]:
exclude_cols = ["claim_number"]

for col in train.columns:
    if col in exclude_cols:
        continue
    uniques = train[col].unique().tolist()
    print(f"\n {col}（共 {len(uniques)} 种取值）:")
    print(uniques[:10])


 subrogation（共 3 种取值）:
[1.0, 0.0, nan]

 year_of_born（共 89 种取值）:
[1990.0, 1972.0, 2003.0, 1983.0, 1985.0, 2002.0, 1965.0, 1967.0, 1958.0, 1987.0]

 gender（共 3 种取值）:
['F', 'M', nan]

 email_or_tel_available（共 3 种取值）:
[0.0, 1.0, nan]

 safety_rating（共 83 种取值）:
[75.0, 94.0, 76.0, 54.0, 95.0, 93.0, 90.0, 79.0, 70.0, 87.0]

 annual_income（共 12118 种取值）:
[70966.0, 79723.0, 41527.0, 42099.0, 47206.0, 88645.0, 76946.0, 57836.0, 38542.0, 117178.0]

 high_education_ind（共 3 种取值）:
[1.0, 0.0, nan]

 address_change_ind（共 3 种取值）:
[1.0, 0.0, nan]

 living_status（共 3 种取值）:
['Rent', 'Own', nan]

 zip_code（共 277 种取值）:
[80040.0, 80030.0, 50012.0, 20138.0, 50033.0, 50005.0, 80027.0, 20148.0, 15009.0, 50050.0]

 claim_date（共 732 种取值）:
['12/4/2016', '4/25/2015', '6/22/2015', '3/2/2015', '1/12/2016', '10/25/2016', '10/4/2015', '10/11/2015', '11/11/2016', '9/17/2015']

 claim_day_of_week（共 8 种取值）:
['Saturday', 'Wednesday', 'Thursday', 'Sunday', 'Monday', 'Tuesday', 'Friday', nan]

 accident_site（共 5 种取值）:
['Pa

In [21]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18001 entries, 0 to 18000
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   subrogation              17999 non-null  float64
 1   claim_number             18001 non-null  int64  
 2   year_of_born             18000 non-null  float64
 3   gender                   18000 non-null  object 
 4   email_or_tel_available   18000 non-null  float64
 5   safety_rating            18000 non-null  float64
 6   annual_income            18000 non-null  float64
 7   high_education_ind       18000 non-null  float64
 8   address_change_ind       18000 non-null  float64
 9   living_status            18000 non-null  object 
 10  zip_code                 18000 non-null  float64
 11  claim_date               18000 non-null  object 
 12  claim_day_of_week        18000 non-null  object 
 13  accident_site            18000 non-null  object 
 14  past_num_of_claims    

In [ ]:
train['claim_number'].nunique() == train.shape[0] # ID列无重复

True

In [19]:
train.duplicated().sum()

0

In [20]:
train.isnull().sum()

subrogation                2
claim_number               0
year_of_born               1
gender                     1
email_or_tel_available     1
safety_rating              1
annual_income              1
high_education_ind         1
address_change_ind         1
living_status              1
zip_code                   1
claim_date                 1
claim_day_of_week          1
accident_site              1
past_num_of_claims         1
witness_present_ind        1
liab_prct                  1
channel                    1
policy_report_filed_ind    1
claim_est_payout           1
vehicle_made_year          1
vehicle_category           1
vehicle_price              1
vehicle_color              1
vehicle_weight             1
age_of_DL                  1
accident_type              1
in_network_bodyshop        1
vehicle_mileage            1
dtype: int64

In [ ]:
mask = (train['policy_report_filed_ind'] == 0) & (train['subrogation'] == 1) # 没有报警，但有subrogation
import pandas as pd

pd.set_option('display.max_columns', None)   # 显示所有列
pd.set_option('display.width', None)         # 自动调整宽度，不换行
pd.set_option('display.max_colwidth', None)  # 不截断列内容

from IPython.display import display
display(train[mask])

#输出后如果想恢复默认设置：
#pd.reset_option('display.max_rows')
#pd.reset_option('display.max_columns')

,subrogation,claim_number,year_of_born,gender,email_or_tel_available,safety_rating,annual_income,high_education_ind,address_change_ind,living_status,zip_code,claim_date,claim_day_of_week,accident_site,past_num_of_claims,witness_present_ind,liab_prct,channel,policy_report_filed_ind,claim_est_payout,vehicle_made_year,vehicle_category,vehicle_price,vehicle_color,vehicle_weight,age_of_DL,accident_type,in_network_bodyshop,vehicle_mileage
13,1.0,2199195,1994.0,M,1.0,90.0,30015.0,0.0,1.0,Own,15025.0,6/10/2016,Wednesday,Parking Area,3.0,Y,33.0,Online,0.0,2863.62,2023.0,Medium,37713.15869,gray,19951.719400,20.0,single_car,yes,125191.0
15,1.0,8209514,1965.0,F,1.0,72.0,33446.0,1.0,0.0,Rent,50008.0,5/3/2016,Thursday,Parking Area,3.0,Y,38.0,Online,0.0,1596.05,2022.0,Medium,15000.00000,black,34027.723010,19.0,multi_vehicle_clear,no,39157.0
19,1.0,8619054,1956.0,F,1.0,48.0,80914.0,0.0,0.0,Own,85027.0,7/4/2015,Tuesday,Parking Area,0.0,Y,35.0,Broker,0.0,5150.61,2022.0,Medium,15000.00000,red,28088.891580,19.0,multi_vehicle_unclear,yes,70287.0
44,1.0,7601282,1988.0,F,1.0,90.0,62709.0,1.0,1.0,Own,50054.0,5/14/2015,Friday,Highway/Intersection,0.0,N,26.0,Online,0.0,711.64,2027.0,Medium,15000.00000,red,26830.410980,22.0,multi_vehicle_unclear,no,56725.0
59,1.0,1178533,1954.0,F,1.0,77.0,41776.0,1.0,0.0,Rent,50041.0,8/28/2016,Tuesday,Parking Area,6.0,N,27.0,Phone,0.0,3797.13,2025.0,Compact,15000.00000,white,11279.211300,22.0,multi_vehicle_unclear,yes,77825.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17922,1.0,6225591,1978.0,F,1.0,58.0,66424.0,1.0,0.0,Rent,20106.0,9/6/2016,Friday,Highway/Intersection,0.0,Y,26.0,Broker,0.0,1313.90,2016.0,Large,21408.37149,blue,22814.435160,25.0,multi_vehicle_unclear,no,58480.0
17943,1.0,7956436,1993.0,F,1.0,53.0,61599.0,1.0,1.0,Rent,85069.0,1/3/2016,Saturday,Highway/Intersection,7.0,N,35.0,Phone,0.0,1164.14,2026.0,Large,15000.00000,red,21963.390550,20.0,multi_vehicle_clear,yes,500.0
17975,1.0,8745480,1994.0,F,0.0,85.0,30015.0,1.0,1.0,Own,15009.0,11/15/2016,Friday,Local,0.0,Y,42.0,Phone,0.0,6796.99,2025.0,Medium,27407.78832,black,61882.954850,16.0,multi_vehicle_clear,yes,58728.0
17977,1.0,2563036,1954.0,F,1.0,83.0,66659.0,1.0,1.0,Rent,85077.0,2/21/2015,Tuesday,Highway/Intersection,0.0,N,33.0,Broker,0.0,889.85,2025.0,Large,15000.00000,blue,33534.029250,25.0,single_car,yes,81183.0


In [14]:
test.head()

,claim_number,year_of_born,gender,email_or_tel_available,safety_rating,annual_income,high_education_ind,address_change_ind,living_status,zip_code,...,claim_est_payout,vehicle_made_year,vehicle_category,vehicle_price,vehicle_color,vehicle_weight,age_of_DL,accident_type,in_network_bodyshop,vehicle_mileage
0,3126034,1987,M,0,68,30015,1,0,Own,85036,...,1373.91,2024,Compact,15000.00000,red,10960.930390,25,single_car,yes,86496
1,7380142,1987,M,1,78,30015,1,0,Rent,20148,...,733.19,2026,Compact,26970.21435,other,13006.313970,22,multi_vehicle_clear,yes,71936
2,4655051,1978,M,1,72,52206,0,1,Rent,20135,...,5885.44,2026,Large,78773.76660,white,10541.156550,24,single_car,yes,84984
3,6728725,1978,M,0,67,52292,0,1,Own,80015,...,860.31,2022,Compact,86166.37419,blue,2733.726122,21,multi_vehicle_clear,yes,51583
4,9848460,1996,M,1,81,41140,0,1,Own,85024,...,4589.91,2024,Medium,15000.00000,gray,18357.025300,18,multi_vehicle_clear,no,108488


In [15]:
train['subrogation'].value_counts()

subrogation
0.0    13884
1.0     4115
Name: count, dtype: int64

In [16]:
import os
print(os.getcwd())  # 当前工作目录
print(os.listdir()) # 当前目录下的文件

/Users/gracewu/Desktop/model-citizens
['.DS_Store', 'travelers.ipynb', '.git', 'data']
